# RNN Training - Flair Labels

Training using the labels generated by Flair.

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
import re

In [24]:
dataset = pd.read_csv('/Users/sankeerthana/Documents/NTU/YEAR_4/SEM_2/CZ4034/IR-Sentiment-Analysis/IR-Sentiment-Analysis/skincare_dataset/crawled_reviews_5k_combined.csv')
dataset.drop(columns='Unnamed: 0', axis=1, inplace=True)
dataset.drop(columns='spacy_labels', axis=1, inplace=True)
dataset.drop(columns='Review_Title', axis=1, inplace=True)
dataset.drop(columns='Product_ID', axis=1, inplace=True)
dataset.head()

,Review,flair_labels
0,From the moment I tried this I fell in love i...,1
1,"After reading the reviews, I decided to buy th...",1
2,I bought this to replace the old Pure (or wha...,1
3,I love this product! I mix one scoop with my ...,1
4,I have a combination skin and like foam cleans...,0


In [25]:
dataset.columns

Index(['Review', 'flair_labels'], dtype='object')

### Data Exploration - Class Balance

In [26]:
dataset['flair_labels'].value_counts()

1    4053
0     947
Name: flair_labels, dtype: int64

From the above, we can say that the model is pretty skewed, and favours the neutral class as opposed to positive or negative. This means we might need to relook into the labels to see if they are actually of 'neutral' nature or add some of the crawled reviews that have the label as 1 and -1.

### Data Processing

In [27]:
#downloading the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sankeerthana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

There are 179 words in the stopwords list provided by NLTK, which we will be using.

In [28]:
def processing_pipeline(data):
    #remove new line characters
    data = re.sub('\s+', ' ', data)

    #remove distracting single quotes
    data = str(re.sub("\'", "", data))

    #converting the text to lowercase
    data = data.lower()
    
    return data
    

In [29]:
#returns a list
def remove_punctuation(data):
    ans = []
    for word in data.split():
        x = word.strip(string.punctuation)
        ans.append(x)

    return ans

In [30]:
#Function to remove stopwords
def remove_stopwords(data):
    #as the input is already a list
    review = data
    filtered_words = []

    for word in review:
        if word not in stopwords.words('english'):
            filtered_words.append(word)

    return filtered_words

In [31]:
#Processing the dataset
for row in dataset.itertuples():
    data = row.Review
   
    #run the processed_pipeline
    data = processing_pipeline(data)

    #remove punctuations
    data = remove_punctuation(data)
    
    #remove stopwords
    data = remove_stopwords(data)

    dataset.at[row.Index,'Review'] = ' '.join(data)

In [32]:
dataset

,Review,flair_labels
0,moment tried fell love everything says continu...,1
1,reading reviews decided buy son dealing random...,1
2,bought replace old pure whatever extra-sensiti...,1
3,love product mix one scoop dear klairs supple ...,1
4,combination skin like foam cleansers started n...,0
...,...,...
4995,reading raving reviews excited try product how...,0
4996,relatively good success deep cystic pimples fe...,0
4997,dissolved make-up made feel like skin could br...,1
4998,decided purchase since already call zero balm ...,1


### Splitting to Train-Test-Validation 

In [33]:
x = dataset['Review']
y = dataset['flair_labels']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [34]:
print(f"Shape of Train: {len(x_train)}")
print(f"Shape of Test: {len(x_test)}")

Shape of Train: 4000
Shape of Test: 1000


### Comverting Sentences to Word Embeddings

In [35]:
MAX_WORDS = 5000
MAX_LEN = 200

tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)




In [36]:
embedding_layer = Embedding(1000, 64)

### Building an RNN

In [37]:
model1 = Sequential()
model1.add(layers.Embedding(MAX_WORDS, 20)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(3,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(x_train, y_train, epochs=70,validation_data=(x_test, y_test),callbacks=[checkpoint1])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-08 11:05:40.143107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-08 11:05:40.144023: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})', "(<class 'list'> containing values of types set())"}), <class 'pandas.core.series.Series'>